In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
#importing necessary packages

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
path = "/content/drive/MyDrive/creditcardAutoencoder.csv"
df = pd.read_csv(path)

In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# the above datset is already PCA transformed and  scaled so i am not distorting it  mean 0 and variance 1

In [7]:
X = df.drop(["Class","Time","Amount"], axis=1).values #dropping redundant columns
y = df["Class"].values

In [8]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (284807, 28)
y shape: (284807,)


In [9]:
from sklearn.model_selection import train_test_split

# Splitting  into train and test 0.2 and startified
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)



Training set shape: (227845, 28)
Test set shape: (56962, 28)


In [15]:
#For training i am  only keeping  normal transactions (Class = 0)
X_train_normal = X_train[y_train == 0]

print("Training set (normal only):", X_train_normal.shape)
print("Fraud cases in training set:", sum(y_train))


# My Autoencoder should only learn what nomral looks like   later on we will measure fraud detection using reconstruction loss
#As my y test has both fraud 1 and non fraud 0,Reconstrcution loss will be higher for the fraud class

Training set (normal only): (227451, 28)
Fraud cases in training set: 394


In [12]:
X_train.shape

(227845, 28)

In [11]:
input_dim = X_train.shape[1]
print(input_dim)


28


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models     #Using functional API sequential not subclassing

# Step 4: Define dimensions
input_dim = X_train.shape[1]   # Its 28 (after dropping Time, Amount, Class)
encoding_dim = 14              #Doing  intermediate compression
bottleneck = 7                 # final latent dimension of bottleneck

# Encoder
encoder = models.Sequential([
    layers.Input(shape=(input_dim,)),          # input layer (28 features)
    layers.Dense(encoding_dim, activation="relu"),
    layers.Dense(bottleneck, activation="relu")  # bottleneck layer
])

# Decoder
decoder = models.Sequential([
    layers.Dense(encoding_dim, activation="relu"),#7>>14
    layers.Dense(input_dim, activation="sigmoid")  # output back to 28 features
])

# Combining Encoder + Decoder
inputs = layers.Input(shape=(input_dim,))
encoded = encoder(inputs)
decoded = decoder(encoded)

autoencoder = models.Model(inputs, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer="adam", loss="mse")

# Summary
autoencoder.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 7)              │           511 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 28)             │           532 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,043 (4.07 KB)

 Trainable params: 1,043 (4.07 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
#Training the Autoencoder
# We train only on normal transactions (Class = 0)
# because the autoencoder should learn to reconstruct "normal" patterns.
# Later, frauds (Class = 1) will have higher reconstruction error.

history = autoencoder.fit(
    X_train_normal,            # input = normal transactions
    X_train_normal,            # output = same (autoencoder reconstructs input)
    epochs=20,                 # train for 20 epochs
    batch_size=256,            # process 256 samples at a time
    validation_split=0.2,      # use 20% of normal data for validation
    shuffle=True,              # shuffle data every epoch
    verbose=1                  # show training progress
)

Epoch 1/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.0778 - val_loss: 0.8801
Epoch 2/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8690 - val_loss: 0.8531
Epoch 3/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8651 - val_loss: 0.8422
Epoch 4/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.8388 - val_loss: 0.8363
Epoch 5/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.8334 - val_loss: 0.8328
Epoch 6/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8357 - val_loss: 0.8295
Epoch 7/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8323 - val_loss: 0.8265
Epoch 8/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8384 - val_loss: 0.8228
Epoch 9/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.8454 - val_loss: 0.8207
Epoch 10/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.8244 - val_loss: 0.8197
Epoch 11/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8383 - val_loss: 0.8187
Epoch 12/20
711/711 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

# Training the autoencoder: Both training and validation losses decrease together,
# showing the model learns normal patterns well without overfitting.

In [18]:
#Reconstruction on normal training data
import numpy as np
reconstructions = autoencoder.predict(X_train_normal)
train_mse = np.mean(np.square(X_train_normal - reconstructions), axis=1)

7108/7108 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step


In [24]:
round(np.mean(train_mse),2)

np.float64(0.82)

In [ ]:
# Our autoencoder is trained only on normal transactions (Class = 0) thats is in X_train normal

# If training MSE is low, it means the model can reconstruct normal data well → confirming normal

# On fraud data (Class = 1), reconstruction error will be much higher, because the model has never seen these patterns.

# Therefore, we set a threshold on reconstruction error (e.g., 95th percentile).

# Any transaction with error above the threshold = anomaly/fraud.

#I used MSE since the data was already PCA and scaled; focus was on reconstruction error, not bringing values back to original scale

In [26]:
# Reconstructing  on test data (both normal + fraud are inside X_test)
reconstructions_test = autoencoder.predict(X_test)

# Computing reconstruction error for each test transaction
test_mse = np.mean(np.square(X_test - reconstructions_test), axis=1)

#Average test MSE
round(np.mean(test_mse),2)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


np.float64(0.87)

In [ ]:
#On test data the mse is slightly higher that is model reconstructs unseen data slighlty worse

In [27]:
# Threshold from training data errors using 95 percentile
threshold_95 = np.percentile(train_mse, 95)
print(f"95th percentile threshold: {threshold_95:.2f}")

95th percentile threshold: 1.79


In [29]:
# Marking  transactions as anomalies if their error > 95th percentile threshold
anomalies = test_mse > threshold_95

# Count how many anomalies
print("Total anomalies detected:", np.sum(anomalies))

Total anomalies detected: 2997


In [30]:
##Comparing with my true labels as y test has both 0 non fraud and 1 fraud

from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
cm = confusion_matrix(y_test, anomalies)
print("Confusion Matrix:\n", cm)

# Detailed precision, recall, f1-score
print("\nClassification Report:\n", classification_report(y_test, anomalies, digits=4))

Confusion Matrix:
 [[53953  2911]
 [   12    86]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9998    0.9488    0.9736     56864
           1     0.0287    0.8776    0.0556        98

    accuracy                         0.9487     56962
   macro avg     0.5142    0.9132    0.5146     56962
weighted avg     0.9981    0.9487    0.9720     56962



In [ ]:
# Fraud recall is great close to 88 percent .We should be more intrested in recall as
# its a anomaly detection where fraud needs to be caught than false alarms


In [31]:
# Saving my  trained autoencoder to Google Drive
autoencoder.save("/content/drive/MyDrive/final_autoencoder_model.h5")
print("Model saved to Drive successfully!")

Model saved to Drive successfully!


Saving my model .Also since the data was already scaled did not have any scaler transform to be saved otherwise would have downladed that too

In [33]:
anomalies = test_mse > threshold_95
import pandas as pd

# Selecting anomalous rows from X_test
anomalies_df = pd.DataFrame(X_test[anomalies], columns=df.drop(["Class", "Time", "Amount"], axis=1).columns)

# Also including  their true labels for evaluation
anomalies_df["True_Class"] = y_test[anomalies]

# Save to CSV
anomalies_df.to_csv("/content/drive/MyDrive/detected_anomalies.csv", index=False)
print("Anomalies saved to CSV in Drive successfully!")


Anomalies saved to CSV in Drive successfully!


INFERENCE

In [34]:
from tensorflow.keras.models import load_model

# Loading the trained autoencoder
autoencoder = load_model("/content/drive/MyDrive/final_autoencoder_model.h5", compile=False) #No need of compile as only inference
print("Model loaded successfully!")

Model loaded successfully!


In [42]:
import numpy as np

# Example: new_data is a numpy array with shape (5, 28)
# (28 because we dropped Time, Amount, Class earlier)
new_data = np.random.randn(5, 28)  # dummy data

In [43]:
reconstructions = autoencoder.predict(new_data) # dummy data

# Compute reconstruction error (MSE per row)
mse = np.mean(np.square(new_data - reconstructions), axis=1)
print("Reconstruction Errors:", mse)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Reconstruction Errors: [0.82040455 0.79197153 1.40175703 0.95591779 0.68419057]


In [46]:
# Using the same threshold i  computed earlier (e.g., 95th percentile = 1.79)
threshold = 1.79 # we may tweak it

# Detect anomalies
anomalies = mse > threshold
print("Anomalies Detected:", anomalies)

Anomalies Detected: [False False False False False]
